In [3]:
import os

In [4]:

%pwd

'd:\\kidney_disease\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\kidney_disease'

In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [23]:
from src.kidney_Disease.constants import *

In [24]:
from src.kidney_Disease.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from src.kidney_Disease import logger
from src.kidney_Disease.utils.common import get_size

In [13]:
%pip install --user --upgrade charset_normalizer

Note: you may need to restart the kernel to use updated packages.


In [1]:
import gdown

In [33]:
class DataIngestion:
  def __init__(self,config:DataIngestionConfig):
    self.config=config
  def download_file(self)->str:
    try:
      dataset_url=self.config.source_URL
      zip_download_dir=self.config.local_data_file
      os.makedirs("artifacts/data_ingestion",exist_ok=True)
      logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")
      file_id = dataset_url.split("/")[-2]
      prefix = 'https://drive.google.com/uc?/export=download&id='
      gdown.download(prefix+file_id,zip_download_dir)

    except Exception as e:
      raise e
  def extract_zip_file(self):
    unzip_path=self.config.unzip_dir
    os.makedirs(unzip_path,exist_ok=True)
    with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
      zip_ref.extractall(unzip_path)  

In [34]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-20 22:46:31,136: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-20 22:46:31,136: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-20 22:46:31,143: INFO: common: created directory at: artifacts]
[2024-07-20 22:46:31,143: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-20 22:46:31,143: INFO: 3416097733: Download data from https://drive.google.com/file/d/1QszaN0FD62iFWMeAROgUclUYlUZd73Bg/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1QszaN0FD62iFWMeAROgUclUYlUZd73Bg
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1QszaN0FD62iFWMeAROgUclUYlUZd73Bg&confirm=t&uuid=cb501547-c4bb-4430-9b3e-c335e9a1c9fa
To: d:\kidney_disease\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:08<00:00, 6.63MB/s]
